In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [4]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts
from vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits

In [5]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.3.0', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [6]:
numpy.set_printoptions(linewidth = 200)

qubits = 5

simulation_backend = "qasm_simulator"

seed = 0
print_circuits = True

In [7]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [8]:
device = provider.get_backend('ibm_cairo')
device

<IBMQBackend('ibm_cairo') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [9]:
device.configuration().to_dict()['max_shots']

100000

In [10]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [11]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [12]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_front.qasm")

In [13]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_mid.qasm")

In [14]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_end.qasm")

In [15]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68324/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [16]:
orign_circuit.draw()

┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────────────■────────»
     ├───────────────┤                                              │        »
q_3: ┤ Ry(-0.019851) ├──────────────────────────────────────────────■────────»
     └───────────────┘                                                       »
q_4: ────────────────────────────────────────────────────────────────────────»
                                                                             »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«     ┌───────────────┐
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘
«q_4: ─────────────────
«

In [17]:
#checking circuit

In [18]:
def apply_checking_circuit(qc, ctrl_bits, ancilla_bits, side = None):
    if len(ctrl_bits) != len(ancilla_bits):
        print("Size mismatch")
        return None
    if side == 'front':
        for i in ancilla_bits:
            qc.h(i)
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
    elif side == 'end':
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
        for i in ancilla_bits:
            qc.h(i)
    else:
        print("Side undefined")

In [19]:
circuit = front_layer.copy()

In [20]:
apply_checking_circuit(circuit, [0], [4], side = 'front')
circuit += mid_layer
apply_checking_circuit(circuit, [0], [4], side = 'end')
circuit += end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68324/1937118134.py:2: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += mid_layer
/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68324/1937118134.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += end_layer


In [21]:
circuit.draw()

┌───┐      ┌───────────────┐            ┌────────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■──■─────■─┤ Ry(-0.0036834) ├»
           ├───┤      ├───────────────┤ │  │     │ ├───────────────┬┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─┼──■──■──┼─┤ Ry(-0.015751) ├─»
     ┌─────┴───┴─────┐└───────────────┘ │     │  │ └───────────────┘ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼─────■──┼─────────■─────────»
     ├───────────────┤                  │        │         │         »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────┼─────────■─────────»
     └─────┬───┬─────┘                  │        │       ┌───┐       »
q_4: ──────┤ H ├────────────────────────■────────■───────┤ H ├───────»
           └───┘                                         └───┘       »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«     ┌───────────────┐
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘
«q_4: ─────────────────
«

In [22]:
pauli_list = read_from_file('LiH_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [23]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 4,num_qargs = 5)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YYYY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
                     
q_3: ────────────────
                     
q_4: ────────────────
                     
ZZYY
                     
q_0: ────────────────
                     
q_1: ────────────────
     ┌──────────────┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZX
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
      ┌────────────┐ 
q_3: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_4: ────────────────
                     
ZXYY
       

In [24]:
circuit_list = []
for meas in meas_qcs:
    temp_circuit = circuit.copy()
    temp_circuit += meas
    circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68324/1717817760.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [25]:
circuit_list[-2].draw()

┌───┐      ┌───────────────┐            ┌────────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■──■─────■─┤ Ry(-0.0036834) ├»
           ├───┤      ├───────────────┤ │  │     │ ├───────────────┬┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─┼──■──■──┼─┤ Ry(-0.015751) ├─»
     ┌─────┴───┴─────┐└───────────────┘ │     │  │ └───────────────┘ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼─────■──┼─────────■─────────»
     ├───────────────┤                  │        │         │         »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────┼─────────■─────────»
     └─────┬───┬─────┘                  │        │       ┌───┐       »
q_4: ──────┤ H ├────────────────────────■────────■───────┤ H ├───────»
           └───┘                                         └───┘       »
«       ┌────────────┐               
«q_0: ──┤ U(π/2,0,π) ├───────────────
«       ├────────────┤               
«q_1: ──┤ U(π/2,0,π) ├───────────────
«     ┌─┴────────────┴┐┌────────────┐
«q_2: ┤ Ry(0.0036205) ├┤ U(π/2,0,π) ├
«     └┬──────────────┤├────────────┤
«q_3: ─┤ Ry(0.019873) ├┤ U(π/2,0,π) ├
«      └──────────────┘└────────────┘
«q_4: ───────────────────────────────
«

In [26]:
# cuts = []
# for i in range(0, 1):
#     if i == 0 or i == qubits - 1:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 6)]
#     else:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 4)]
#     cuts += cut

In [27]:
cuts = [(circuit.qubits[0], 3),(circuit.qubits[0], 4)]

In [28]:
fragments_list = []
wire_path_map_list = []
for temp_qc in circuit_list:
    print(temp_qc)
    fragments, wire_path_map = circuit_cutter.cut_circuit(temp_qc, cuts)
    fragments_list.append(fragments)
    wire_path_map_list.append(wire_path_map)

           ┌───┐      ┌───────────────┐            ┌────────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■──■─────■─┤ Ry(-0.0036834) ├»
           ├───┤      ├───────────────┤ │  │     │ ├───────────────┬┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─┼──■──■──┼─┤ Ry(-0.015751) ├─»
     ┌─────┴───┴─────┐└───────────────┘ │     │  │ └───────────────┘ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼─────■──┼─────────■─────────»
     ├───────────────┤                  │        │         │         »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────┼─────────■─────────»
     └─────┬───┬─────┘                  │        │       ┌───┐       »
q_4: ──────┤ H ├────────────────────────■────────■───────┤ H ├───────»
           └───┘                                         └───┘       »
«      ┌──────────────┐                
«q_0: ─┤ U(π/2,0,π/2) ├────────────────
«      ├──────────────┤                
«q_1: ─┤ U(π/2,0,π/2) ├────────────────
«     ┌┴──────────────┤┌──────────────┐
«q_2: ┤ Ry(0.003620

           ┌───┐      ┌───────────────┐            ┌────────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■──■─────■─┤ Ry(-0.0036834) ├»
           ├───┤      ├───────────────┤ │  │     │ ├───────────────┬┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─┼──■──■──┼─┤ Ry(-0.015751) ├─»
     ┌─────┴───┴─────┐└───────────────┘ │     │  │ └───────────────┘ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼─────■──┼─────────■─────────»
     ├───────────────┤                  │        │         │         »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────┼─────────■─────────»
     └─────┬───┬─────┘                  │        │       ┌───┐       »
q_4: ──────┤ H ├────────────────────────■────────■───────┤ H ├───────»
           └───┘                                         └───┘       »
«       ┌────────────┐ 
«q_0: ──┤ U(π/2,0,π) ├─
«       ├────────────┤ 
«q_1: ──┤ U(π/2,0,π) ├─
«     ┌─┴────────────┴┐
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘
«q_

           ┌───┐      ┌───────────────┐            ┌────────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■──■─────■─┤ Ry(-0.0036834) ├»
           ├───┤      ├───────────────┤ │  │     │ ├───────────────┬┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─┼──■──■──┼─┤ Ry(-0.015751) ├─»
     ┌─────┴───┴─────┐└───────────────┘ │     │  │ └───────────────┘ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼─────■──┼─────────■─────────»
     ├───────────────┤                  │        │         │         »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────┼─────────■─────────»
     └─────┬───┬─────┘                  │        │       ┌───┐       »
q_4: ──────┤ H ├────────────────────────■────────■───────┤ H ├───────»
           └───┘                                         └───┘       »
«       ┌────────────┐               
«q_0: ──┤ U(π/2,0,π) ├───────────────
«       ├────────────┤               
«q_1: ──┤ U(π/2,0,π) ├───────────────
«     ┌─┴────────────┴┐┌────────────┐
«q_2: ┤ Ry(0.0036205) ├┤ U(π/

In [29]:
print(wire_path_map_list[0])

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(3, 'q'), 1)), (1, Qubit(QuantumRegister(4, 'q'), 0)), (0, Qubit(QuantumRegister(3, 'q'), 2))), Qubit(QuantumRegister(5, 'q'), 1): ((1, Qubit(QuantumRegister(4, 'q'), 2)),), Qubit(QuantumRegister(5, 'q'), 2): ((1, Qubit(QuantumRegister(4, 'q'), 1)),), Qubit(QuantumRegister(5, 'q'), 3): ((1, Qubit(QuantumRegister(4, 'q'), 3)),), Qubit(QuantumRegister(5, 'q'), 4): ((0, Qubit(QuantumRegister(3, 'q'), 0)),)}


In [30]:
total_variants = ml.fragment_variants(wire_path_map_list[0])

In [31]:
total_variants

24

In [32]:
shots = device.configuration().to_dict()['max_shots'] * total_variants

In [33]:
fragments_list[0][0].draw()

┌───┐                             ┌───┐                       
q_0: ┤ H ├──────────────────■──■───────┤ H ├───────────────────────
     ├───┤┌───────────────┐ │  │       └───┘                       
q_1: ┤ X ├┤ Ry(-0.003534) ├─■──┼───────────────────────────────────
     └───┘└───────────────┘    │ ┌────────────────┐┌──────────────┐
q_2: ──────────────────────────■─┤ Ry(-0.0036834) ├┤ U(π/2,0,π/2) ├
                                 └────────────────┘└──────────────┘

In [34]:
fragment_cuts_list = []
for index in range(0, len(fragments_list)):
    for idx, fragment in enumerate(fragments_list[index]):
        print(f"fragment {idx}:")
        print(fragment)
        print()
    fragment_cuts = ml.fragment_cuts(wire_path_map_list[index])
    fragment_cuts_list.append(fragment_cuts)


fragment 0:
     ┌───┐                             ┌───┐                       
q_0: ┤ H ├──────────────────■──■───────┤ H ├───────────────────────
     ├───┤┌───────────────┐ │  │       └───┘                       
q_1: ┤ X ├┤ Ry(-0.003534) ├─■──┼───────────────────────────────────
     └───┘└───────────────┘    │ ┌────────────────┐┌──────────────┐
q_2: ──────────────────────────■─┤ Ry(-0.0036834) ├┤ U(π/2,0,π/2) ├
                                 └────────────────┘└──────────────┘

fragment 1:
                                                                 »
q_0: ───────────────────────────────────■────────────────────────»
     ┌───────────────┐                  │                        »
q_1: ┤ Ry(0.0038243) ├──────────────────┼──■───────────────────■─»
     └─────┬───┬─────┘┌───────────────┐ │  │ ┌───────────────┐ │ »
q_2: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─┤ Ry(-0.015751) ├─┼─»
     ┌─────┴───┴─────┐└───────────────┘      └───────────────┘ │ »
q_3: ┤ Ry(-0.019851) ├────────

In [35]:
print(fragment_cuts_list[-1])

[{'prep': 1, 'meas': 1}, {'prep': 1, 'meas': 1}]


In [36]:
wire_path_map

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(3, 'q'), 1)),
  (1, Qubit(QuantumRegister(4, 'q'), 0)),
  (0, Qubit(QuantumRegister(3, 'q'), 2))),
 Qubit(QuantumRegister(5, 'q'), 1): ((1, Qubit(QuantumRegister(4, 'q'), 2)),),
 Qubit(QuantumRegister(5, 'q'), 2): ((1, Qubit(QuantumRegister(4, 'q'), 1)),),
 Qubit(QuantumRegister(5, 'q'), 3): ((1, Qubit(QuantumRegister(4, 'q'), 3)),),
 Qubit(QuantumRegister(5, 'q'), 4): ((0, Qubit(QuantumRegister(3, 'q'), 0)),)}

In [37]:
num_fragments = len(fragments)
num_fragments

2

In [43]:
num_tomo_circuits = len(meas_qcs)
num_tomo_circuits

25

In [44]:
fragments[0].draw()

┌───┐                             ┌───┐       
q_0: ┤ H ├──────────────────■──■───────┤ H ├───────
     ├───┤┌───────────────┐ │  │       └───┘       
q_1: ┤ X ├┤ Ry(-0.003534) ├─■──┼───────────────────
     └───┘└───────────────┘    │ ┌────────────────┐
q_2: ──────────────────────────■─┤ Ry(-0.0036834) ├
                                 └────────────────┘

In [45]:
fragments[1].draw()

»
q_0: ───────────────────────────────────■────────────────────────»
     ┌───────────────┐                  │                        »
q_1: ┤ Ry(0.0038243) ├──────────────────┼──■───────────────────■─»
     └─────┬───┬─────┘┌───────────────┐ │  │ ┌───────────────┐ │ »
q_2: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─┤ Ry(-0.015751) ├─┼─»
     ┌─────┴───┴─────┐└───────────────┘      └───────────────┘ │ »
q_3: ┤ Ry(-0.019851) ├─────────────────────────────────────────■─»
     └───────────────┘                                           »
«                      
«q_0: ─────────────────
«     ┌───────────────┐
«q_1: ┤ Ry(0.0036205) ├
«     └───────────────┘
«q_2: ─────────────────
«      ┌──────────────┐
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘

In [46]:
hardware_index = 1

In [47]:
tomography_circuits = []
for idx in range(0, len(fragments_list)):
    fragments = fragments_list[idx]
    wire_path_map = wire_path_map_list[idx]
    temp_frag_circuits = [0] * len(fragments)
    for i in range(0, len(fragments)):
        if i == hardware_index:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend, initial_layout = [23,25,24,26], opt_lvl = 3, extra_qc = None)
        else:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
    tomography_circuits.append(temp_frag_circuits)

In [48]:
#contains 25 basis
len(tomography_circuits)

25

In [49]:
#contains three fragments
len(tomography_circuits[0])

2

In [50]:
len(tomography_circuits[0][1])

1

In [51]:
#tomography circuits
len(tomography_circuits[0][1][0])

12

In [52]:
multi_exp_circuits = [[],[]]
for i in range(0, len(tomography_circuits)):
    #for each basis collect the three fragments:
    for j in range(0, num_fragments):
        fragment_circ = tomography_circuits[i][j][0]
        multi_exp_circuits[j] += fragment_circ

In [53]:
multi_exp_circuits[1][-200].draw()

┌─────────────────┐ ░                                         »
  q_0: ┤ U3(-1.9106,π,0) ├─░────────────────────────────────────■────»
       └─────────────────┘ ░ ┌───────────────┐                  │    »
  q_1: ────────────────────░─┤ Ry(0.0038243) ├──────────────────┼──■─»
                           ░ └─────┬───┬─────┘┌───────────────┐ │  │ »
  q_2: ────────────────────░───────┤ X ├──────┤ Ry(-0.015869) ├─■──■─»
                           ░ ┌─────┴───┴─────┐└───────────────┘      »
  q_3: ────────────────────░─┤ Ry(-0.019851) ├───────────────────────»
                           ░ └───────────────┘                       »
c17_0: ══════════════════════════════════════════════════════════════»
                                                                     »
c17_1: ══════════════════════════════════════════════════════════════»
                                                                     »
c17_2: ══════════════════════════════════════════════════════════════»
                                                                     »
c17_3: ══════════════════════════════════════════════════════════════»
                                                                     »
«                                                           ░ ┌───┐      ┌─┐
«  q_0: ────────────────────────────────────────────────────░─┤ H ├──────┤M├
«                           ┌───────────────┐┌────────────┐ ░ └┬─┬┘      └╥┘
«  q_1: ──────────────────■─┤ Ry(0.0036205) ├┤ U(π/2,0,π) ├─░──┤M├────────╫─
«       ┌───────────────┐ │ └┬──────────────┤└────────────┘ ░  └╥┘ ┌─┐    ║ 
«  q_2: ┤ Ry(-0.015751) ├─┼──┤ U(π/2,0,π/2) ├───────────────░───╫──┤M├────╫─
«       └───────────────┘ │  ├──────────────┤┌────────────┐ ░   ║  └╥┘┌─┐ ║ 
«  q_3: ──────────────────■──┤ Ry(0.019873) ├┤ U(π/2,0,π) ├─░───╫───╫─┤M├─╫─
«                            └──────────────┘└────────────┘ ░   ║   ║ └╥┘ ║ 
«c17_0: ════════════════════════════════════════════════════════╬═══╬══╬══╩═
«                                                               ║   ║  ║    
«c17_1: ════════════════════════════════════════════════════════╩═══╬══╬════
«                                                                   ║  ║    
«c17_2: ════════════════════════════════════════════════════════════╩══╬════
«                                                                      ║    
«c17_3: ═══════════════════════════════════════════════════════════════╩════
«

In [55]:
frag_datas = [0] * num_fragments
for i in range(0, num_fragments):
    if i == hardware_index:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = [23,25,24,26], 
                                     backend = device, shots = shots // total_variants, 
                                     optimization_level = 3, monitor_jobs = True)
    else:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = None,
                                     backend = simulation_backend, shots = shots // total_variants,
                                     optimization_level = 3, monitor_jobs = True, noise_model = None)
       

job_id:  25d66422-95d8-4904-9b6c-407e0a47dfcd
Job Status: job has successfully run
job_id:  6377111bb95e160aefc27866
Job Status: job is queued (124)   

IBMQJobApiError: '"HTTPSConnectionPool(host=\'api-qcon.quantum-computing.ibm.com\', port=443): Max retries exceeded with url: /api/Network/ibm-q-ornl/Groups/ornl/Projects/chm185/Jobs/6377111bb95e160aefc27866/status/v/1 (Caused by NewConnectionError(\'<urllib3.connection.HTTPSConnection object at 0x7fa26ade6eb0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known\'))"'

In [56]:
frag_datas[0] = run_circuits(multi_exp_circuits[0], initial_layout = None,
                             backend = simulation_backend, shots = shots // total_variants,
                             optimization_level = 3, monitor_jobs = True, noise_model = None)

job_id:  9e4ba6c1-0193-4293-a31d-75cc3ce62bcd
Job Status: job has successfully run


In [60]:
frag_datas[1] = [device.retrieve_job(job_id='6377111bb95e160aefc27866').result()]

In [61]:
frag_datas[0][0].results[0].header.name

"(('S0', 'S0', 'S0'), ('Z', 'Z', 'Z'))"

In [62]:
frag_datas[1][0].results[0].header.name

"(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z'))"

In [63]:
num_tomo_circuits

25

In [64]:
test = [[], []]

In [70]:
results_list = [[], []]
counts_list = [[], []] 
for i in range(0, num_fragments): 
    for j in range(0, num_tomo_circuits):
        start_idx = j * 12
        end_idx = (j + 1) * 12
        results_list[i].append(frag_datas[i][0].results[start_idx : end_idx])
        counts_list[i].append(frag_datas[i][0].get_counts()[start_idx : end_idx])


In [71]:
frag_datas[0][0].get_counts()

[{'011': 50204, '111': 49796},
 {'001': 24918, '111': 25115, '011': 24696, '101': 25271},
 {'111': 25282, '001': 25134, '101': 24790, '011': 24794},
 {'111': 16826, '110': 33330, '010': 33200, '011': 16644},
 {'011': 8248,
  '111': 8328,
  '001': 8443,
  '110': 16498,
  '100': 16544,
  '000': 16583,
  '010': 16914,
  '101': 8442},
 {'110': 16506,
  '001': 8370,
  '100': 16621,
  '010': 16757,
  '000': 16543,
  '111': 8405,
  '011': 8334,
  '101': 8464},
 {'111': 16686, '011': 16607, '010': 33598, '110': 33109},
 {'011': 8275,
  '111': 8393,
  '010': 16715,
  '000': 16636,
  '101': 8547,
  '001': 8443,
  '110': 16422,
  '100': 16569},
 {'111': 8188,
  '011': 8377,
  '101': 8359,
  '001': 8228,
  '100': 16765,
  '110': 16829,
  '010': 16539,
  '000': 16715},
 {'111': 16766, '011': 16719, '110': 33258, '010': 33257},
 {'111': 8383,
  '011': 8313,
  '000': 16620,
  '010': 16613,
  '101': 8311,
  '001': 8443,
  '110': 16665,
  '100': 16652},
 {'101': 8355,
  '011': 8382,
  '111': 8242,
  '1

In [72]:
frag_datas[1][0].get_counts()

[{'0000': 3822,
  '0001': 61,
  '0010': 4226,
  '0011': 75,
  '0100': 3817,
  '0101': 71,
  '0110': 4013,
  '0111': 79,
  '1000': 3559,
  '1001': 68,
  '1010': 4268,
  '1011': 53,
  '1100': 3626,
  '1101': 73,
  '1110': 4129,
  '1111': 60},
 {'0000': 1914,
  '0001': 1823,
  '0010': 2272,
  '0011': 2139,
  '0100': 1840,
  '0101': 1896,
  '0110': 2092,
  '0111': 2161,
  '1000': 1916,
  '1001': 1839,
  '1010': 2217,
  '1011': 2079,
  '1100': 1868,
  '1101': 1877,
  '1110': 2102,
  '1111': 1965},
 {'0000': 1836,
  '0001': 2096,
  '0010': 2043,
  '0011': 2282,
  '0100': 1839,
  '0101': 1834,
  '0110': 1943,
  '0111': 2129,
  '1000': 1795,
  '1001': 1995,
  '1010': 2071,
  '1011': 2266,
  '1100': 1779,
  '1101': 1881,
  '1110': 2039,
  '1111': 2172},
 {'0000': 1417,
  '0001': 2447,
  '0010': 1560,
  '0011': 2582,
  '0100': 1281,
  '0101': 2612,
  '0110': 1387,
  '0111': 2960,
  '1000': 1360,
  '1001': 2345,
  '1010': 1467,
  '1011': 2602,
  '1100': 1205,
  '1101': 2522,
  '1110': 1392,
  '11

In [76]:
import ast, itertools, numpy, scipy, qiskit, tensornetwork
import qiskit.ignis.verification as verif
from qiskit.tools import monitor
from qiskit.providers.ibmq.managed import IBMQJobManager
from qiskit import IBMQ, transpile


prep_state_keys = { "Pauli" : [ "Zp", "Zm", "Xp", "Yp" ],
                    "SIC" : [ "S0", "S1", "S2", "S3" ] }
meas_state_keys = { "Pauli" : [ "Zp", "Zm", "Xp", "Xm", "Yp", "Ym" ],
                    "SIC" : [ "S0", "S1", "S2", "S3" ] }


def organize_tomography_data_from_list(raw_results_list, counts_list, prep_qubits, meas_qubits, prep_basis, extra_qc):
    def _qubit_index(qubit):
        if type(qubit) is int: return qubit
        else: return qubit.index
    prep_qubits = list(map(_qubit_index, prep_qubits))
    meas_qubits = list(map(_qubit_index, meas_qubits))

    # split a bitstring on all qubits into:
    #   (1) a bitstring on the "middle" qubits that are associated with a cut, and
    #   (2) a bitstring on the "final" qubits that are *not* associated with a cut
    def _split_bits(bits):
        mid_bits = "".join([ bits[len(bits)-idx-1] for idx in meas_qubits ])
        fin_bits = "".join([ bit for pos, bit in enumerate(bits)
                             if len(bits)-pos-1 not in meas_qubits ])
        return mid_bits, fin_bits

    organized_data = {}
    if len(extra_qc) == 0:
        appending_size = None
    else:
        appending_size = -len(extra_qc)
    for index in range(0, len(raw_results_list)):
        name = raw_results_list[index].header.name
        meas_counts = counts_list[index]
        full_prep_label, full_meas_label = ast.literal_eval(name)
        print(name, full_prep_label, full_meas_label)

        prep_label = tuple( full_prep_label[qubit] for qubit in prep_qubits )
        meas_label = tuple( full_meas_label[qubit] for qubit in meas_qubits )
        print(name, prep_label, meas_label, prep_qubits, meas_qubits)
        for bits, counts in meas_counts.items():
            meas_bits, final_bits = _split_bits(bits)
            meas_state = tuple( basis + ( "p" if outcome == "0" else "m" )
                                for basis, outcome in zip(meas_label, meas_bits) )
            count_label = ( prep_label, meas_state )
            if final_bits not in organized_data:
                organized_data[final_bits] = {}
            organized_data[final_bits][count_label] = counts

    # add zero count data for output strings with missing prep/meas combinations
    prep_labels = itertools.product(prep_state_keys[prep_basis], repeat = len(prep_qubits))
    meas_states = itertools.product(meas_state_keys["Pauli"], repeat = len(meas_qubits))
    count_labels = list(itertools.product(prep_labels, meas_states))
    for bits in organized_data.keys():
        if len(organized_data[bits]) == len(count_labels): continue
        for count_label in count_labels:
            if count_label not in organized_data[bits]:
                organized_data[bits][count_label] = 0

    return organized_data


In [78]:
organized_data = []
for i in range(0, num_tomo_circuits):
    frag_data = []
    frag_targets = ml.identify_frag_targets(wire_path_map_list[i])
    for j in range(0, num_fragments):
        frag_data.append(organize_tomography_data_from_list(results_list[j][i], counts_list[j][i],
                                              frag_targets[j].get("prep"),
                                              frag_targets[j].get("meas"),
                                              prep_basis = "SIC", extra_qc = []))
    organized_data.append(frag_data)

(('S0', 'S0', 'S0'), ('Z', 'Z', 'Z')) ('S0', 'S0', 'S0') ('Z', 'Z', 'Z')
(('S0', 'S0', 'S0'), ('Z', 'Z', 'Z')) ('S0',) ('Z',) [2] [1]
(('S0', 'S0', 'S0'), ('Z', 'X', 'Z')) ('S0', 'S0', 'S0') ('Z', 'X', 'Z')
(('S0', 'S0', 'S0'), ('Z', 'X', 'Z')) ('S0',) ('X',) [2] [1]
(('S0', 'S0', 'S0'), ('Z', 'Y', 'Z')) ('S0', 'S0', 'S0') ('Z', 'Y', 'Z')
(('S0', 'S0', 'S0'), ('Z', 'Y', 'Z')) ('S0',) ('Y',) [2] [1]
(('S0', 'S0', 'S1'), ('Z', 'Z', 'Z')) ('S0', 'S0', 'S1') ('Z', 'Z', 'Z')
(('S0', 'S0', 'S1'), ('Z', 'Z', 'Z')) ('S1',) ('Z',) [2] [1]
(('S0', 'S0', 'S1'), ('Z', 'X', 'Z')) ('S0', 'S0', 'S1') ('Z', 'X', 'Z')
(('S0', 'S0', 'S1'), ('Z', 'X', 'Z')) ('S1',) ('X',) [2] [1]
(('S0', 'S0', 'S1'), ('Z', 'Y', 'Z')) ('S0', 'S0', 'S1') ('Z', 'Y', 'Z')
(('S0', 'S0', 'S1'), ('Z', 'Y', 'Z')) ('S1',) ('Y',) [2] [1]
(('S0', 'S0', 'S2'), ('Z', 'Z', 'Z')) ('S0', 'S0', 'S2') ('Z', 'Z', 'Z')
(('S0', 'S0', 'S2'), ('Z', 'Z', 'Z')) ('S2',) ('Z',) [2] [1]
(('S0', 'S0', 'S2'), ('Z', 'X', 'Z')) ('S0', 'S0', 'S2') ('Z',

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68324/3788665908.py:17: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  else: return qubit.index


In [79]:
for final_bits, fixed_bit_data in frag_data[0].items():
    prep_meas_states, state_counts = zip(*fixed_bit_data.items())
    prep_labels, meas_labels = zip(*prep_meas_states)
    prep_qubit_num = len(prep_labels[0])
    meas_qubit_num = len(meas_labels[0])
    print(prep_qubit_num)

1
1
1
1


In [80]:
len(organized_data)

25

In [81]:
organized_data[2]

[{'11': {(('S0',), ('Zm',)): 1,
   (('S2',), ('Zp',)): 2,
   (('S2',), ('Xp',)): 1,
   (('S3',), ('Xp',)): 1,
   (('S0',), ('Zp',)): 0,
   (('S0',), ('Xp',)): 0,
   (('S0',), ('Xm',)): 0,
   (('S0',), ('Yp',)): 0,
   (('S0',), ('Ym',)): 0,
   (('S1',), ('Zp',)): 0,
   (('S1',), ('Zm',)): 0,
   (('S1',), ('Xp',)): 0,
   (('S1',), ('Xm',)): 0,
   (('S1',), ('Yp',)): 0,
   (('S1',), ('Ym',)): 0,
   (('S2',), ('Zm',)): 0,
   (('S2',), ('Xm',)): 0,
   (('S2',), ('Yp',)): 0,
   (('S2',), ('Ym',)): 0,
   (('S3',), ('Zp',)): 0,
   (('S3',), ('Zm',)): 0,
   (('S3',), ('Xm',)): 0,
   (('S3',), ('Yp',)): 0,
   (('S3',), ('Ym',)): 0},
  '00': {(('S0',), ('Zp',)): 2,
   (('S1',), ('Zp',)): 1,
   (('S2',), ('Zm',)): 1,
   (('S0',), ('Zm',)): 0,
   (('S0',), ('Xp',)): 0,
   (('S0',), ('Xm',)): 0,
   (('S0',), ('Yp',)): 0,
   (('S0',), ('Ym',)): 0,
   (('S1',), ('Zm',)): 0,
   (('S1',), ('Xp',)): 0,
   (('S1',), ('Xm',)): 0,
   (('S1',), ('Yp',)): 0,
   (('S1',), ('Ym',)): 0,
   (('S2',), ('Zp',)): 0,

In [82]:
organized_data[-1][-1]

{'000': {(('S0',), ('Zp',)): 934,
  (('S0',), ('Zm',)): 28,
  (('S0',), ('Xp',)): 525,
  (('S0',), ('Xm',)): 488,
  (('S0',), ('Yp',)): 518,
  (('S0',), ('Ym',)): 545,
  (('S1',), ('Zp',)): 371,
  (('S1',), ('Zm',)): 652,
  (('S1',), ('Xp',)): 433,
  (('S1',), ('Xm',)): 652,
  (('S1',), ('Yp',)): 443,
  (('S1',), ('Ym',)): 633,
  (('S2',), ('Zp',)): 351,
  (('S2',), ('Zm',)): 612,
  (('S2',), ('Xp',)): 469,
  (('S2',), ('Xm',)): 549,
  (('S2',), ('Yp',)): 647,
  (('S2',), ('Ym',)): 428,
  (('S3',), ('Zp',)): 325,
  (('S3',), ('Zm',)): 723,
  (('S3',), ('Xp',)): 622,
  (('S3',), ('Xm',)): 428,
  (('S3',), ('Yp',)): 433,
  (('S3',), ('Ym',)): 579},
 '001': {(('S0',), ('Zp',)): 42,
  (('S0',), ('Zm',)): 4,
  (('S0',), ('Xp',)): 25,
  (('S0',), ('Xm',)): 24,
  (('S0',), ('Yp',)): 24,
  (('S0',), ('Ym',)): 26,
  (('S1',), ('Zp',)): 20,
  (('S1',), ('Zm',)): 29,
  (('S1',), ('Xp',)): 20,
  (('S1',), ('Xm',)): 21,
  (('S1',), ('Yp',)): 10,
  (('S1',), ('Ym',)): 24,
  (('S2',), ('Zp',)): 14,
 

In [101]:
recombined_lists = []
filtered_recombined_lists = []
norm_filtered_lists = []
for i in range(0, num_tomo_circuits):
    direct_models = ml.direct_fragment_model(organized_data[i])
    likely_models = ml.maximum_likelihood_model(direct_models)

    direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map_list[i])
    likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map_list[i])
    recombined_lists.append(likely_recombined_dist)
    filtered_recombined_lists.append(filter_results(likely_recombined_dist, [0]))
    norm_filtered_lists.append(norm_dict(filter_results(likely_recombined_dist, [0])))

{'01': {(('S0',), ('Zm',)): 50204, (('S0',), ('Xp',)): 24918, (('S0',), ('Xm',)): 24696, (('S0',), ('Yp',)): 25134, (('S0',), ('Ym',)): 24794, (('S1',), ('Zm',)): 16644, (('S1',), ('Xm',)): 8248, (('S1',), ('Xp',)): 8443, (('S1',), ('Yp',)): 8370, (('S1',), ('Ym',)): 8334, (('S2',), ('Zm',)): 16607, (('S2',), ('Xm',)): 8275, (('S2',), ('Xp',)): 8443, (('S2',), ('Ym',)): 8377, (('S2',), ('Yp',)): 8228, (('S3',), ('Zm',)): 16719, (('S3',), ('Xm',)): 8313, (('S3',), ('Xp',)): 8443, (('S3',), ('Ym',)): 8382, (('S3',), ('Yp',)): 8447, (('S0',), ('Zp',)): 0, (('S1',), ('Zp',)): 0, (('S2',), ('Zp',)): 0, (('S3',), ('Zp',)): 0}, '11': {(('S0',), ('Zm',)): 49796, (('S0',), ('Xm',)): 25115, (('S0',), ('Xp',)): 25271, (('S0',), ('Ym',)): 25282, (('S0',), ('Yp',)): 24790, (('S1',), ('Zm',)): 16826, (('S1',), ('Xm',)): 8328, (('S1',), ('Xp',)): 8442, (('S1',), ('Ym',)): 8405, (('S1',), ('Yp',)): 8464, (('S2',), ('Zm',)): 16686, (('S2',), ('Xm',)): 8393, (('S2',), ('Xp',)): 8547, (('S2',), ('Ym',)):

{'000': {(('S0',), ('Zp',)): 241, (('S0',), ('Zm',)): 10, (('S0',), ('Xp',)): 126, (('S0',), ('Xm',)): 142, (('S0',), ('Yp',)): 124, (('S0',), ('Ym',)): 157, (('S1',), ('Zp',)): 84, (('S1',), ('Zm',)): 191, (('S1',), ('Xp',)): 107, (('S1',), ('Xm',)): 177, (('S1',), ('Yp',)): 133, (('S1',), ('Ym',)): 178, (('S2',), ('Zp',)): 91, (('S2',), ('Zm',)): 173, (('S2',), ('Xp',)): 147, (('S2',), ('Xm',)): 132, (('S2',), ('Yp',)): 180, (('S2',), ('Ym',)): 116, (('S3',), ('Zp',)): 87, (('S3',), ('Zm',)): 191, (('S3',), ('Xp',)): 160, (('S3',), ('Xm',)): 125, (('S3',), ('Yp',)): 117, (('S3',), ('Ym',)): 147}, '001': {(('S0',), ('Zp',)): 256, (('S0',), ('Zm',)): 10, (('S0',), ('Xp',)): 154, (('S0',), ('Xm',)): 153, (('S0',), ('Yp',)): 121, (('S0',), ('Ym',)): 139, (('S1',), ('Zp',)): 84, (('S1',), ('Zm',)): 188, (('S1',), ('Xp',)): 110, (('S1',), ('Xm',)): 144, (('S1',), ('Yp',)): 119, (('S1',), ('Ym',)): 177, (('S2',), ('Zp',)): 69, (('S2',), ('Zm',)): 175, (('S2',), ('Xp',)): 127, (('S2',), ('Xm

{'000': {(('S0',), ('Zp',)): 506, (('S0',), ('Zm',)): 15, (('S0',), ('Xp',)): 266, (('S0',), ('Xm',)): 269, (('S0',), ('Yp',)): 256, (('S0',), ('Ym',)): 249, (('S1',), ('Zp',)): 179, (('S1',), ('Zm',)): 371, (('S1',), ('Xp',)): 214, (('S1',), ('Xm',)): 318, (('S1',), ('Yp',)): 240, (('S1',), ('Ym',)): 317, (('S2',), ('Zp',)): 204, (('S2',), ('Zm',)): 322, (('S2',), ('Xp',)): 274, (('S2',), ('Xm',)): 295, (('S2',), ('Yp',)): 332, (('S2',), ('Ym',)): 214, (('S3',), ('Zp',)): 152, (('S3',), ('Zm',)): 373, (('S3',), ('Xp',)): 278, (('S3',), ('Xm',)): 225, (('S3',), ('Yp',)): 251, (('S3',), ('Ym',)): 304}, '001': {(('S0',), ('Zp',)): 468, (('S0',), ('Zm',)): 14, (('S0',), ('Xp',)): 262, (('S0',), ('Xm',)): 290, (('S0',), ('Yp',)): 279, (('S0',), ('Ym',)): 256, (('S1',), ('Zp',)): 190, (('S1',), ('Zm',)): 351, (('S1',), ('Xp',)): 256, (('S1',), ('Xm',)): 312, (('S1',), ('Yp',)): 226, (('S1',), ('Ym',)): 329, (('S2',), ('Zp',)): 216, (('S2',), ('Zm',)): 376, (('S2',), ('Xp',)): 274, (('S2',),

{'000': {(('S0',), ('Zp',)): 7636, (('S0',), ('Zm',)): 138, (('S0',), ('Xp',)): 3875, (('S0',), ('Xm',)): 3879, (('S0',), ('Yp',)): 3712, (('S0',), ('Ym',)): 4117, (('S1',), ('Zp',)): 2662, (('S1',), ('Zm',)): 5351, (('S1',), ('Xp',)): 532, (('S1',), ('Xm',)): 7557, (('S1',), ('Yp',)): 3040, (('S1',), ('Ym',)): 5111, (('S2',), ('Zp',)): 3001, (('S2',), ('Zm',)): 4821, (('S2',), ('Xp',)): 4941, (('S2',), ('Xm',)): 2870, (('S2',), ('Yp',)): 7269, (('S2',), ('Ym',)): 436, (('S3',), ('Zp',)): 2515, (('S3',), ('Zm',)): 5314, (('S3',), ('Xp',)): 6241, (('S3',), ('Xm',)): 1521, (('S3',), ('Yp',)): 1405, (('S3',), ('Ym',)): 6455}, '001': {(('S0',), ('Zp',)): 322, (('S0',), ('Zm',)): 7, (('S0',), ('Xp',)): 169, (('S0',), ('Xm',)): 153, (('S0',), ('Yp',)): 169, (('S0',), ('Ym',)): 178, (('S1',), ('Zp',)): 119, (('S1',), ('Zm',)): 223, (('S1',), ('Xp',)): 22, (('S1',), ('Xm',)): 321, (('S1',), ('Yp',)): 116, (('S1',), ('Ym',)): 215, (('S2',), ('Zp',)): 134, (('S2',), ('Zm',)): 188, (('S2',), ('Xp

{'000': {(('S0',), ('Zp',)): 4225, (('S0',), ('Zm',)): 74, (('S0',), ('Xp',)): 2248, (('S0',), ('Xm',)): 2129, (('S0',), ('Yp',)): 2114, (('S0',), ('Ym',)): 2331, (('S1',), ('Zp',)): 1522, (('S1',), ('Zm',)): 3040, (('S1',), ('Xp',)): 320, (('S1',), ('Xm',)): 4144, (('S1',), ('Yp',)): 1628, (('S1',), ('Ym',)): 2815, (('S2',), ('Zp',)): 1613, (('S2',), ('Zm',)): 2656, (('S2',), ('Xp',)): 2620, (('S2',), ('Xm',)): 1650, (('S2',), ('Yp',)): 4088, (('S2',), ('Ym',)): 239, (('S3',), ('Zp',)): 1391, (('S3',), ('Zm',)): 2875, (('S3',), ('Xp',)): 3465, (('S3',), ('Xm',)): 840, (('S3',), ('Yp',)): 812, (('S3',), ('Ym',)): 3495}, '001': {(('S0',), ('Zp',)): 3548, (('S0',), ('Zm',)): 73, (('S0',), ('Xp',)): 1795, (('S0',), ('Xm',)): 1775, (('S0',), ('Yp',)): 1712, (('S0',), ('Ym',)): 1917, (('S1',), ('Zp',)): 1257, (('S1',), ('Zm',)): 2487, (('S1',), ('Xp',)): 287, (('S1',), ('Xm',)): 3485, (('S1',), ('Yp',)): 1467, (('S1',), ('Ym',)): 2446, (('S2',), ('Zp',)): 1432, (('S2',), ('Zm',)): 2264, (('

In [85]:
direct_recombined_dist

{'10000': 0.0020236179609372075,
 '10100': -6.602061283748674e-06,
 '10010': 0.02297044280537021,
 '10110': -0.00016151310003266738,
 '11000': -9.885205565823257e-05,
 '11100': -6.931497527593603e-08,
 '11010': -9.214593110962371e-05,
 '11110': -6.927821009698218e-05,
 '10001': 1.9113640359667717e-07,
 '10101': 3.507014141041592e-09,
 '10011': 1.5201229454599476e-05,
 '10111': 6.235991119355856e-07,
 '11001': 9.783173567119526e-09,
 '11101': 2.4120612648447798e-09,
 '11011': 2.5021600005177083e-07,
 '11111': 1.6858720537487585e-08,
 '00000': 6.591229318746613e-08,
 '00100': 8.902121787121521e-09,
 '00010': -1.3071834443489448e-05,
 '00110': -5.049718011612442e-07,
 '01000': -1.0557198705388594e-09,
 '01100': -7.926226395109571e-10,
 '01010': -1.9743487314962825e-07,
 '01110': -1.9336330401299578e-08,
 '00001': 0.030855868792690654,
 '00101': 0.001070370506871996,
 '00011': 0.8892066519007407,
 '00111': 0.03678428398469112,
 '01001': 0.0006520681051869594,
 '01101': 7.829389423493818e-0

In [86]:
likely_recombined_dist

{'10000': 0.001962573228074433,
 '10100': 4.7504484355497046e-05,
 '10010': 0.024010101651417814,
 '10110': 6.607773859060027e-05,
 '11000': 4.2196829140915174e-05,
 '11100': 1.1621722807745743e-05,
 '11010': 0.00022272634752096559,
 '11110': 2.5754397369270492e-05,
 '00001': 0.030741426983992785,
 '00101': 0.0009926446170381382,
 '00011': 0.8879410063182296,
 '00111': 0.036666329255427815,
 '01001': 0.000584695719369483,
 '01101': 3.14079439736694e-05,
 '01011': 0.015581120451517563,
 '01111': 0.001072812311173786}

In [87]:
filtered_recombined_lists

[{'0001': 0.05529526845116151,
  '0101': 0.060955545372026694,
  '0011': 0.06102913103062592,
  '0111': 0.06742482922191936,
  '1001': 0.05510008522229474,
  '1101': 0.062023917207571375,
  '1011': 0.056859058789119045,
  '1111': 0.06585604619633188,
  '0000': 0.055442172013947834,
  '0100': 0.06114170353777217,
  '0010': 0.06120014558723067,
  '0110': 0.06760946425225876,
  '1000': 0.055242681921452416,
  '1100': 0.06220111193284451,
  '1010': 0.05700675166687563,
  '1110': 0.06604058630718083},
 {'0000': 0.22342667596874735,
  '0100': 0.008462061126385933,
  '0010': 0.23842236187956092,
  '0110': 0.008645391282972875,
  '1000': 0.003278028037969216,
  '1010': 0.003356948426793905,
  '0001': 0.2244507094406943,
  '0101': 0.008502976349263931,
  '0011': 0.2393987729429009,
  '0111': 0.008682888462307702,
  '1001': 0.003293359342227827,
  '1011': 0.003374773478262992},
 {'0001': 0.008130121129584627,
  '0101': 0.008187101711102958,
  '0011': 0.22289621013025762,
  '0111': 0.247673202673

In [88]:
from utils.utils import filter_results

In [89]:
filter_direct_recombined = filter_results(direct_recombined_dist, [0])

In [90]:
filter_direct_recombined

{'0000': 6.591229318746613e-08,
 '0100': 8.902121787121521e-09,
 '0010': -1.3071834443489448e-05,
 '0110': -5.049718011612442e-07,
 '1000': -1.0557198705388594e-09,
 '1100': -7.926226395109571e-10,
 '1010': -1.9743487314962825e-07,
 '1110': -1.9336330401299578e-08,
 '0001': 0.030855868792690654,
 '0101': 0.001070370506871996,
 '0011': 0.8892066519007407,
 '0111': 0.03678428398469112,
 '1001': 0.0006520681051869594,
 '1101': 7.829389423493818e-05,
 '1011': 0.01568002669292439,
 '1111': 0.0011042578989443254}

In [91]:

filter_likely_recombined = filter_results(likely_recombined_dist, [0])

In [92]:
filter_likely_recombined

{'0001': 0.030741426983992785,
 '0101': 0.0009926446170381382,
 '0011': 0.8879410063182296,
 '0111': 0.036666329255427815,
 '1001': 0.000584695719369483,
 '1101': 3.14079439736694e-05,
 '1011': 0.015581120451517563,
 '1111': 0.001072812311173786}

In [93]:
def norm_dict(dictionary):
    total = total_counts(dictionary)
    norm_dist = {}
    for i in dictionary.keys():
        norm_dist[i] = dictionary[i]/total
    return norm_dist

In [94]:
norm_filter_dist = norm_dict(filter_likely_recombined)

In [95]:
likely_recombined_dist['00011']

0.8879410063182296

In [96]:
#normalized distribution after mitigation: 0.9189375->0.9286796265616319
norm_filter_dist['0011']

0.9120075695026171

In [97]:
norm_filter_dist

{'0001': 0.031574636048135665,
 '0101': 0.0010195490445007761,
 '0011': 0.9120075695026171,
 '0111': 0.03766012560393101,
 '1001': 0.0006005431871333531,
 '1101': 3.225921817179233e-05,
 '1011': 0.016003427808832706,
 '1111': 0.001101889586677604}

In [102]:
for i in range(0, len(filtered_recombined_lists)):
    print(total_counts(norm_filtered_lists[i]))

1.0000000000000002
1.0000000000000002
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999998
1.0
1.0
0.9999999999999999
1.0000000000000002
0.9999999999999999
1.0
1.0000000000000002
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999


In [103]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(norm_filtered_lists[i], shots = 1, Pauli = Pauli_tuple[0])  

In [ ]:
-1.08

In [104]:
final_expect_val

(-1.023653274098594+0j)

In [107]:
len(norm_filtered_lists)

25

In [106]:
import json

In [108]:
with open('LiH_cairo_check0.json', 'w') as f:
    json.dump(norm_filtered_lists, f)